In [1]:
import sqlite3
import pandas as pd
from glob import glob

In [2]:
# データベースに接続（ファイルがなければ作成される）
conn = sqlite3.connect("../db/pachislo.db")
cursor = conn.cursor()

# 2. 新しいテーブル（UNIQUE付き）を作成
cursor.execute("""
CREATE TABLE IF NOT EXISTS slot_data (
    date TEXT,
    hall_id INTEGER,
    model TEXT,
    unit_no INTEGER,
    start INTEGER,
    bb INTEGER,
    rb INTEGER,
    art INTEGER,
    max_medals INTEGER,
    bb_rate TEXT,
    rb_rate TEXT,
    art_rate TEXT,
    total_rate TEXT,
    last_start INTEGER
)
""")

conn.commit()

for row in conn.execute("PRAGMA table_info(slot_data)"):
    print(row)

(0, 'date', 'TEXT', 0, None, 0)
(1, 'hall_id', 'INTEGER', 0, None, 0)
(2, 'model', 'TEXT', 0, None, 0)
(3, 'unit_no', 'INTEGER', 0, None, 0)
(4, 'start', 'INTEGER', 0, None, 0)
(5, 'bb', 'INTEGER', 0, None, 0)
(6, 'rb', 'INTEGER', 0, None, 0)
(7, 'art', 'INTEGER', 0, None, 0)
(8, 'max_medals', 'INTEGER', 0, None, 0)
(9, 'bb_rate', 'TEXT', 0, None, 0)
(10, 'rb_rate', 'TEXT', 0, None, 0)
(11, 'art_rate', 'TEXT', 0, None, 0)
(12, 'total_rate', 'TEXT', 0, None, 0)
(13, 'last_start', 'INTEGER', 0, None, 0)


In [ ]:
csv_list = glob("../csv/*.csv")
for csv in csv_list:
    date = csv.replace("../csv\\EXA_FIRST_SﾏｲｼﾞｬｸﾞﾗｰⅤKD_", "").replace(".csv", "")
    df = pd.read_csv(csv)
    df["date"] = date
    df["hall_id"] = 101262
    df["model"] = "SﾏｲｼﾞｬｸﾞﾗｰⅤKD"
    df_sqlite = df.rename(columns={
        "台番号": "unit_no", "累計スタート": "start", "BB回数": "bb", "RB回数": "rb",
        "ART回数": "art", "最大持玉": "max_medals", "BB確率": "bb_rate", "RB確率": "rb_rate",
        "ART確率": "art_rate", "合成確率": "total_rate", "前日最終スタート": "last_start"
    })
    df_sqlite.to_sql("slot_data", conn, if_exists="append", index=False)

In [4]:
# データベースに接続（ファイルがなければ作成される）
conn = sqlite3.connect("../db/pachislo.db")
cursor = conn.cursor()

# データ読み込み
query = "SELECT * FROM slot_data"
df = pd.read_sql(query, conn)

conn.close()


In [ ]:
df_pivot = df.drop_duplicates(subset=["unit_no", "date"])
df_pivot = df_pivot.pivot(index="unit_no", columns="date", values="total_rate")